In [ ]:
# tại cell code này thì sẽ tạo ra submission chưa qua post-processing, điểm đạt được là 0.257
import pandas as pd
import numpy as np
import glob
import os

# 1. Tự động tìm đường dẫn file trong Dataset input
def find_sub_file(keyword):
    files = glob.glob(f'/kaggle/input/**/*{keyword}*.tsv', recursive=True)
    if not files:
        raise FileNotFoundError(f"Không tìm thấy file chứa từ khóa '{keyword}'")
    return files[0]

try:
    path_mlp = find_sub_file('mlp')
    path_cnn = find_sub_file('cnn')
    print(f" -> Found MLP: {path_mlp}")
    print(f" -> Found CNN: {path_cnn}")
except Exception as e:
    print(f"LỖI: {e}")
    raise

# 2. Đọc dữ liệu
print(" -> Reading files...")
df_mlp = pd.read_csv(path_mlp, sep='\t', header=None, names=['id', 'term', 'score'])
df_cnn = pd.read_csv(path_cnn, sep='\t', header=None, names=['id', 'term', 'score'])

print(f"   + MLP Shape: {df_mlp.shape}")
print(f"   + CNN Shape: {df_cnn.shape}")

# 3. Trộn (Merge)
print(" -> Merging (Outer Join)...")
df_blend = pd.merge(df_mlp, df_cnn, on=['id', 'term'], how='outer', suffixes=('_mlp', '_cnn'))
# Điền 0 vào những chỗ model này đoán ra mà model kia không đoán
df_blend = df_blend.fillna(0)

# 4. Tính điểm trung bình có trọng số (Weighted Average)
print(" -> Calculating Weighted Score...")

# --- TRỌNG SỐ ---
W_MLP = 0.6
W_CNN = 0.4

df_blend['score'] = (df_blend['score_mlp'] * W_MLP) + (df_blend['score_cnn'] * W_CNN)

# 5. Lọc và Lưu file cuối cùng
print(" -> Formatting & Saving final submission.tsv...")

# Lọc ngưỡng > 0.01
df_final = df_blend[df_blend['score'] > 0.01][['id', 'term', 'score']]

# Format 3 số thập phân chuẩn CAFA
df_final['score'] = df_final['score'].map('{:.3f}'.format)

# Sắp xếp
df_final = df_final.sort_values(by=['id', 'score'], ascending=[True, False])

# Ghi file
df_final.to_csv('submission.tsv', sep='\t', header=False, index=False)

print(f"DONE!{len(df_final)} dòng.")

In [ ]:
#nâng điểm lên 0.261
import pandas as pd
from collections import defaultdict
import os
from tqdm import tqdm

# --- CẤU HÌNH ĐƯỜNG DẪN ---
INPUT_SUBMISSION = '/kaggle/input/cafa6-ensemble-source/submission.tsv' #đường dẫn file submission sẽ postprocessing
OBO_FILE = '/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo'
OUTPUT_FILE = 'submission.tsv'

# --- PHẦN 1: HÀM ĐỌC CẤU TRÚC OBO ---
def parse_go_obo(obo_path):
    print(f"Loading OBO from: {obo_path} ...")
    go_parents = defaultdict(set)
    current_id = None
    
    # Mở file và đọc từng dòng
    with open(obo_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line.startswith("id: GO:"):
                current_id = line.split("id: ")[1]
            elif line.startswith("is_a: GO:") and current_id:
                parent = line.split("is_a: ")[1].split(" !")[0]
                go_parents[current_id].add(parent)
            elif line.startswith("relationship: part_of GO:") and current_id:
                parent = line.split("relationship: part_of ")[1].split(" !")[0]
                go_parents[current_id].add(parent)
                
    print(f"-> Đã load xong quan hệ cha-con của {len(go_parents)} GO terms.")
    return go_parents

# --- PHẦN 2: HÀM TÌM TỔ TIÊN (ĐỆ QUY CÓ CACHE) ---
def get_ancestors(go_term, go_parents, cache):
    # Nếu đã tính rồi thì trả về ngay (Memoization)
    if go_term in cache:
        return cache[go_term]
    
    parents = set()
    # Lấy cha trực tiếp
    direct_parents = go_parents.get(go_term, [])
    
    for p in direct_parents:
        parents.add(p)
        # Đệ quy để lấy ông, cụ, kỵ...
        parents.update(get_ancestors(p, go_parents, cache))
        
    cache[go_term] = parents
    return parents

# --- PHẦN 3: LOGIC LAN TRUYỀN (PROPAGATION) ---
def propagate_scores(df, go_parents):
    print("Bắt đầu lan truyền điểm số (Propagation)...")
    
    # Chuyển DataFrame thành Dictionary để xử lý cho nhanh
    # Cấu trúc: data[protein_id] = {go_term: score, ...}
    data = defaultdict(dict)
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Reading CSV"):
        data[row[0]][row[1]] = float(row[2]) # row[0]=ID, row[1]=Term, row[2]=Score
    
    final_rows = []
    ancestor_cache = {} # Cache để không phải tìm lại tổ tiên nhiều lần
    
    # Duyệt qua từng Protein
    for protein_id, terms_dict in tqdm(data.items(), desc="Propagating"):
        # Copy ra dict mới để chứa điểm sau khi lan truyền
        propagated_dict = terms_dict.copy()
        
        # Duyệt qua từng term hiện có của protein đó
        for go_term, original_score in terms_dict.items():
            # Lấy tất cả tổ tiên của term này
            ancestors = get_ancestors(go_term, go_parents, ancestor_cache)
            
            # Cập nhật điểm cho tổ tiên
            for ancestor in ancestors:
                current_score = propagated_dict.get(ancestor, 0.0)
                # Logic Max: Điểm tổ tiên = Max(Điểm cũ của nó, Điểm của con cháu)
                if original_score > current_score:
                    propagated_dict[ancestor] = original_score
        
        # Lưu lại kết quả vào list
        for term, score in propagated_dict.items():
            if score >= 0.001: # Lọc nhẹ bớt các điểm quá thấp để giảm dung lượng
                final_rows.append((protein_id, term, score))
                
    return final_rows

# --- MAIN ---
if __name__ == "__main__":
    # 1. Đọc dữ liệu
    if not os.path.exists(OBO_FILE):
        print("Lỗi")
        exit()
        
    go_parents = parse_go_obo(OBO_FILE)
    
    print(f"Reading submission: {INPUT_SUBMISSION}")
    df = pd.read_csv(INPUT_SUBMISSION, sep='\t', header=None, names=['Id', 'Term', 'Score'])
    
    # 2. Xử lý
    result_data = propagate_scores(df, go_parents)
    
    # 3. Xuất file
    print(f"Saving to {OUTPUT_FILE}...")
    result_df = pd.DataFrame(result_data, columns=['Id', 'Term', 'Score'])
    
    # Format float 3
    result_df.to_csv(OUTPUT_FILE, sep='\t', header=False, index=False, float_format='%.3f')